In [72]:
import pandas as pd
from streamlit_assets.components import get_recos_users
from sklearn.preprocessing import LabelEncoder
from surprise import SVD, NMF, KNNBasic, SVDpp
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import Reader
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

### Exploring metacritic data for recos

In [11]:
meta_df = pd.read_csv('data/metacritic.csv').dropna(subset='critic_meta_score')
#meta_df = meta_df.rename(columns={"title":"item_id"})

meta_df.head()


,title,rank,critic_meta_score,review_content,review_source,author,review_date,summary,meta_score,user_score,release_date,link,critics_reviews_link
0,Rectify: Season 4,1,100.0,It allows us to know and care for these charac...,Collider,Allison Keene,"Oct 27, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
1,Rectify: Season 4,1,100.0,"Rectify, a drama entering its final season on ...",The New York Times,James Poniewozik,"Oct 25, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
2,Rectify: Season 4,1,100.0,No other series so poignantly probes the human...,Salon,Melanie McFarland,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
3,Rectify: Season 4,1,100.0,None of these characters is particularly happy...,Yahoo TV,Ken Tucker,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
4,Rectify: Season 4,1,100.0,Rectify is the best series I have ever seen on...,The Daily Beast,Malcolm Jones,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...


In [38]:
meta_df['title_group'] = meta_df['title'].str.split(': Season').str[0]
meta_df.head()

,title,rank,critic_meta_score,review_content,review_source,author,review_date,summary,meta_score,user_score,release_date,link,critics_reviews_link,title_group,user_id,item_id,rating
0,Rectify: Season 4,1,100.0,It allows us to know and care for these charac...,Collider,Allison Keene,"Oct 27, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify,58,Rectify: Season 4,100
1,Rectify: Season 4,1,100.0,"Rectify, a drama entering its final season on ...",The New York Times,James Poniewozik,"Oct 25, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify,576,Rectify: Season 4,100
2,Rectify: Season 4,1,100.0,No other series so poignantly probes the human...,Salon,Melanie McFarland,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify,1025,Rectify: Season 4,100
3,Rectify: Season 4,1,100.0,None of these characters is particularly happy...,Yahoo TV,Ken Tucker,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify,803,Rectify: Season 4,100
4,Rectify: Season 4,1,100.0,Rectify is the best series I have ever seen on...,The Daily Beast,Malcolm Jones,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify,945,Rectify: Season 4,100


In [39]:
id_maker = LabelEncoder()
meta_df['user_id'] = id_maker.fit_transform(meta_df['author'])
meta_df['item_id'] = meta_df['title']
meta_df['rating'] = meta_df['critic_meta_score'].apply(int)

reviews = meta_df[['user_id','item_id','rating']]

In [47]:
tv_list = ['Breaking Bad: Season 5',
           'Rectify: Season 4',
           'Better Call Saul: Season 5',
           'The Leftovers: Season 1',
           'Lodge 49',
           'The Underground Railroad',
           'Master of None: Season 1',
           'Normal People: Season 1',
           'Fleabag: Season 1'
          ]


In [63]:
df_recos = get_recos_users(tv_list,reviews, filter_items=True,filter_n=10, default_rating=90)

C:\git\reco_system\streamlit_assets\components.py:82: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_reviews = df.append(new_df).reset_index(drop=True)


In [64]:
df_merged = meta_df.merge(df_recos, on='item_id').drop_duplicates(subset='item_id')
df_merged['difference'] = df_merged['user match'] - df_merged['meta_score']
df_merged = df_merged.sort_values(by='user match', ascending=False).reset_index(drop=True)
df_merged[['title','summary','user match','meta_score','difference']].head(50).sort_values(by='user match', ascending = False)

,title,summary,user match,meta_score,difference
0,Better Call Saul: Season 6,The sixth and final season of the Breaking Bad...,95.024701,94,1.024701
1,The Handmaid's Tale: Season 1,"In the totalitarian society of Gilead, a handm...",94.242487,92,2.242487
2,Atlanta: Season 2,The second season subtitled the Atlanta Robbin...,94.147231,97,-2.852769
3,Louie: Season 4,"After 19 months, the Emmy-winning comedy retur...",93.791409,93,0.791409
4,Atlanta: Season 3,"Earn, Alfred, Darius and Van are on tour in Eu...",93.245025,91,2.245025
5,The Americans: Season 5,The espionage drama was renewed for two season...,93.101516,94,-0.898484
6,The Leftovers: Season 3,The third and final season of the Damon Lindel...,93.085462,98,-4.914538
7,The Americans: Season 4,Elizabeth and Phillips are given a new dangero...,92.821630,95,-2.178370
8,It's a Sin,The five-part miniseries written and created b...,92.746325,91,1.746325
9,Barry: Season 3,Barry finds it's not just other people stoppin...,92.410778,94,-1.589222


In [67]:
df_grouped = df_merged.groupby('title_group').mean().sort_values(by='user match', ascending = False)


In [70]:
df_grouped[['user match','meta_score','difference']].head(50)

,user match,meta_score,difference
title_group,,,
It's a Sin,92.746325,91.000000,1.746325
City So Real,92.247868,93.000000,-0.752132
The Beatles: Get Back,92.149175,85.000000,7.149175
Fleabag,91.724099,96.000000,-4.275901
Severance,91.258447,83.000000,8.258447
Homecoming: A Film by Beyoncé,90.407554,93.000000,-2.592446
Unbelievable,90.381903,83.000000,7.381903
Atlanta,90.342956,92.666667,-2.323711
Frozen Planet,89.932915,91.000000,-1.067085


In [71]:
df_grouped.plot(x='user match', y='meta_score', kind='scatter')

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.